In [1]:
from read_write_files import read_json, save_json, get_parser_paths
from collections import Counter
import itertools
import pandas as pd
import numpy as np
import conll16st.scorer as scorer

In [2]:
def create_sense_zip(alignments):
    attr = "Sense"
    attr_zip = []
    for alignment in alignments:
        line_attr = []
        line_attr += [alignment["gold"][attr][0]]
        
        for pars in alignment["parsers"]:
            if pars == None:
                line_attr += ["None"]
            else:
                line_attr += [pars[attr][0]]
        
        attr_zip += [tuple(line_attr)]
    return attr_zip,alignment["parser_names"]

In [3]:
def sense_statistics(sense_comparison,parser_names,not_mapped_rels):
    zip_sense_comparison = list(zip(*sense_comparison))
    gold_senses = zip_sense_comparison[0]
    #Count all combination of senses
    sense_counter = Counter(sense_comparison)
    #Different Senses
    set_senses = set(gold_senses)
    #How many parsers will be compared
    len_parser = len(zip_sense_comparison)-1
    sense_rows = []
    
    #not mappable predicted relations
    not_mapped_counter = Counter([(rel["Parser"],rel["Sense"][0]) for rel in not_mapped_rels])

    for sense in set_senses:
        tmp_senses = set_senses.copy()
        tmp_senses.remove(sense)
        possible_comb = [[sense]] + [tmp_senses for i in range(len_parser)]
        diff_preds = itertools.product(*possible_comb)

        equal_correct_parsing = sense_counter[tuple([sense]+[sense]*len_parser)]

        all_wrong_parsing = sum([
            sense_counter[tuple(diff_pred)] 
            for diff_pred in diff_preds])

        total_act_sense_count = sum([1 for i in gold_senses if i == sense]) # + equal_correct_parsing 


        equal_wrong_parsing = sum([
            sense_counter[tuple([sense]+[other_sense]*len_parser)] 
            for other_sense in tmp_senses])
        
        parser_columns = []

        #Parser is better than all the other
        parser_better = []
        for index_parser in range(1,len_parser+1):
            tmp_possible_comb = possible_comb[:]
            column_name = parser_names[index_parser-1]
            
            tmp_possible_comb[index_parser] = [sense]
            other_diff_preds = itertools.product(*tmp_possible_comb)
            
            pars_better = sum([sense_counter[other_diff_pred] for other_diff_pred in other_diff_preds])
            total_pred_sense_count = Counter(zip_sense_comparison[index_parser])[sense]#sum([1 for i in zip_sense_comparison[index_parser] if i == sense])
            
            not_mapped = not_mapped_counter[(column_name,sense)]
            
            right_pred_comb = [[sense]] + [set_senses]*len_parser
            right_pred_comb[index_parser] = [sense]
            right_pred = sum([sense_counter[comb] for comb in itertools.product(*right_pred_comb)])
            
            parser_better += [
                pars_better,
                not_mapped,
                right_pred,
                total_pred_sense_count+not_mapped]
            
            
            parser_columns += [string.format(column_name) for string in [
                "{}: unique right",
                "{}: not mapped",
                "{}: right pred",
                "{}: total pred"]]
            
        #Two Parser share opinions
        for parser1,parser2 in itertools.combinations([i for i in range(1,len_parser+1)],2):
            tmp_possible_comb = possible_comb[:]
            tmp_possible_comb[parser1] = [sense]
            tmp_possible_comb[parser2] = [sense]
            
            
            pars_better = sum([sense_counter[comb] for comb in itertools.product(*tmp_possible_comb)])
            total_pred_sense_count = sum(
                [1 for pars1_pred,pars2_pred in zip(*[
                    zip_sense_comparison[parser1],
                    zip_sense_comparison[parser2]
                ])
                if (pars2_pred == sense) != (pars1_pred == sense)])
            
            parser_better += [pars_better,total_pred_sense_count]
            
            column_name = " and ".join([parser_names[parser1-1],parser_names[parser2-1]])
            parser_columns += [
                "{} right".format(column_name),
                "Total Pred ({})".format(column_name)]
        

        #At least one parser is correct
        correct_sense_poss = [[sense]]+[set_senses]*len_parser
        combination_correct = sum(
            [sense_counter[at_least_one_right] 
             for at_least_one_right in itertools.product(*correct_sense_poss) if sense in at_least_one_right[1:]])

        sense_rows += [[sense,equal_correct_parsing,all_wrong_parsing,total_act_sense_count,equal_wrong_parsing,combination_correct]+parser_better]

    columns = ["Sense",
             "Equal Correct",
             "All Wrong",
             "Total Act",
             "Equal Wrong",
             "At least one correct"]+parser_columns
    return pd.DataFrame(
        columns=columns,
        data=sense_rows),columns


In [87]:
def sense_statistics2(sense_comparison,parser_names,not_mapped_rels):
    zip_sense_comparison = list(zip(*sense_comparison))
    gold_senses = zip_sense_comparison[0]
    #Count all combination of senses
    sense_counter = Counter(sense_comparison)
    #Different Senses
    set_senses = set(gold_senses)
    #How many parsers will be compared
    len_parser = len(zip_sense_comparison)-1
    sense_rows = []
    
    #not mappable predicted relations
    not_mapped_counter = Counter([(rel["Parser"],rel["Sense"][0]) for rel in not_mapped_rels])
    
    parser_comb = list(itertools.combinations([i for i in range(1,len_parser+1)],2))
    
    for sense in set_senses:
    
        equal_correct_parsing = 0
        all_wrong_parsing = 0
        total_act_sense_count = 0
        equal_wrong_parsing = 0
        
        individ_parser_stats = {
            parser_index:[
                0,
                0,
                0,
                0
            ] for parser_index in range(1,len_parser+1)
        }
        double_parser_stats = {
            parsers:[0,0]
            for parsers in parser_comb
        }

        combination_correct = 0
        
        for sense_comb in sense_comparison:
            if sense_comb[0] == sense:
                total_act_sense_count += 1
                if tuple([sense]+[sense]*len_parser) == sense_comb:
                    equal_correct_parsing += 1
                if not sense in sense_comb[1:]:
                    all_wrong_parsing += 1
                    if len(set(list(sense_comb[1:]))) == 1:
                        equal_wrong_parsing += 1
                else:
                    combination_correct += 1
                for parser_index in range(1,len_parser+1):
                    parser_pred = sense_comb[parser_index] 
                    other_preds = [pred for ind,pred in enumerate(sense_comb[1:]) if ind+1 != parser_index]
                    if parser_pred == sense:
                        individ_parser_stats[parser_index][2] += 1
                        if sense not in other_preds:
                            individ_parser_stats[parser_index][0] += 1
                            
                    individ_parser_stats[parser_index][3] += 1
                    
                for p_comb in parser_comb:
                    if sense_comb[p_comb[0]] == sense and sense_comb[p_comb[1]] == sense:
                        double_parser_stats[p_comb][0] += 1
                        
                    if sense_comb[p_comb[0]] == sense or sense_comb[p_comb[1]] == sense:
                        double_parser_stats[p_comb][1] += 1
                        
        for parser_index in range(1,len_parser+1):                
            individ_parser_stats[parser_index][1] = not_mapped_counter[
                (parser_names[parser_index-1],sense)]
        
        sense_row = [[sense,
             equal_correct_parsing,
             all_wrong_parsing,
             total_act_sense_count,
             equal_wrong_parsing,
             combination_correct]]+[
                individ_parser_stats[parser_index]
                for parser_index in range(1,len_parser+1)
            ]+[
                double_parser_stats[parsers]
                for parsers in parser_comb
            ]
        
        sense_row = [measure for part in sense_row for measure in part]
        
        sense_rows += [sense_row]

    
    columns = ["Sense",
         "Equal Correct",
         "All Wrong",
         "Total Act",
         "Equal Wrong",
         "At least one correct"
    ]+[
        string.format(parser_name) for parser_name in parser_names 
        for string in [
            "{}: unique right",
            "{}: not mapped",
            "{}: right pred",
            "{}: total pred"]
        
    ]+[
        string.format(parser_names[ind1-1],parser_names[ind2-1]) for ind1,ind2 in parser_comb
        for string in ["{}/{}: both right","{}/{}: total pred"]
        
    ]


    return pd.DataFrame(
        columns=columns,
        data=sense_rows),columns


In [79]:
alignment_path = "data/project_files/test/total_alignment.json"
not_mappable_path = "data/project_files/test/not_mappable.json"

best3_alignment_path = "data/project_files/test/3best_alignment.json"
best3_not_mappable_path = "data/project_files/test/3best_not_mappable.json"

total_alignments = read_json(alignment_path)
not_mappable = read_json(not_mappable_path)

best3_alignments = read_json(best3_alignment_path)
best3_not_mappable = read_json(best3_not_mappable_path)

In [11]:
sense_zip,sense_zip_names = create_sense_zip(total_alignments)
sense_counter = Counter(sense_zip)

In [88]:
best3_sense_zip,best3_sense_zip_names = create_sense_zip(best3_alignments)
best3_sense_counter = Counter(best3_sense_zip)

sense_df,df_columns = sense_statistics2(best3_sense_zip,best3_sense_zip_names,best3_not_mappable)

In [89]:
print("Overall Statistics")
sense_df[df_columns[:6]]

Overall Statistics


,Sense,Equal Correct,All Wrong,Total Act,Equal Wrong,At least one correct
0,Comparison,0,1,1,0,0
1,Expansion.Conjunction,103,64,391,13,327
2,Expansion.Instantiation,14,35,91,4,56
3,Temporal.Asynchronous.Precedence,19,7,45,0,38
4,Comparison.Concession,3,19,33,4,14
5,Contingency.Cause.Result,15,75,135,5,60
6,Contingency.Cause.Reason,27,75,195,10,120
7,Expansion,0,3,3,0,0
8,Expansion.Alternative,0,0,5,0,5
9,Temporal.Asynchronous.Succession,15,24,69,14,45


In [90]:
print("Parser Individual Statistics")
sense_df[df_columns[6:6+len(sense_zip_names)*4]]

Parser Individual Statistics


,steven: unique right,steven: not mapped,steven: right pred,steven: total pred,oslopots: unique right,oslopots: not mapped,oslopots: right pred,oslopots: total pred,ecnucs: unique right,ecnucs: not mapped,ecnucs: right pred,ecnucs: total pred,steven/oslopots: both right,steven/oslopots: total pred,steven/ecnucs: both right,steven/ecnucs: total pred,oslopots/ecnucs: both right,oslopots/ecnucs: total pred
0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
1,20,1,178,391,41,1,264,391,23,3,231,391,138,304,123,286,188,307
2,1,0,17,91,5,0,33,91,21,0,49,91,15,35,15,51,27,55
3,1,0,26,45,2,2,34,45,1,0,31,45,23,37,21,36,28,37
4,1,0,5,33,1,0,12,33,1,0,11,33,4,13,3,13,10,13
5,5,1,33,135,5,0,42,135,13,0,37,135,28,47,15,55,24,55
6,4,6,42,195,25,1,92,195,22,2,82,195,36,98,29,95,58,116
7,0,0,0,3,0,0,0,3,0,0,0,3,0,0,0,0,0,0
8,0,0,2,5,1,0,4,5,0,0,3,5,1,5,1,4,2,5
9,2,2,25,69,3,3,41,69,1,1,33,69,22,44,16,42,31,43


In [91]:
print("Two Parser Agreement")
sense_df[df_columns[-len(best3_sense_zip_names)*2:]]

Two Parser Agreement


,steven/oslopots: both right,steven/oslopots: total pred,steven/ecnucs: both right,steven/ecnucs: total pred,oslopots/ecnucs: both right,oslopots/ecnucs: total pred
0,0,0,0,0,0,0
1,138,304,123,286,188,307
2,15,35,15,51,27,55
3,23,37,21,36,28,37
4,4,13,3,13,10,13
5,28,47,15,55,24,55
6,36,98,29,95,58,116
7,0,0,0,0,0,0
8,1,5,1,4,2,5
9,22,44,16,42,31,43


# Model Training

In [13]:
def get_sense_lists(relations):
    gold_senses = []
    parser_senses = []
    parser_names = relations[0]["parser_names"]
    
    for rel in relations:
        gold_senses += [rel["gold"]["Sense"][0]]
        
        parser_pred = []
        for parser in rel["parsers"]:
            if parser == None:
                parser_pred += ["None"]
            else:
                parser_pred += [parser["Sense"][0]]
        
        parser_senses += tuple([parser_pred])
    parser_senses_zip = list(zip(*parser_senses))
    
    return gold_senses,parser_senses_zip,parser_names
        
            
    

In [14]:
def calc_prob_measure(gold_total,pred_total,tp):
    prec = 0
    if pred_total != 0:
        prec = tp/pred_total
        
    rec = 0
    if gold_total != 0:
        rec = tp/gold_total
    
    f1 = 0
    if prec != 0 and rec != 0:
        f1 = ((prec*rec)/(prec+rec))
    
    return prec,rec,f1

In [15]:
def calc_sense_category_probs(sense_model,senses):
    
    sense_split = [(sense.split("."),sense_model[sense]) for sense in senses]
    
    new_senses = {}
    ready = True
    while ready:
        for index,(splitted_sense,f1) in enumerate(sense_split):
            ready = False
            if len(splitted_sense) > 1:
                ready = True
                new_sense = ".".join(splitted_sense[:-1])
                if new_sense in new_senses:
                    new_senses[new_sense] += [sense_model[".".join(splitted_sense)]]
                else:
                    new_senses[new_sense] = [sense_model[".".join(splitted_sense)]]
                sense_split[index][0] = 
        
    return sense_model

SyntaxError: invalid syntax (<ipython-input-15-9aa89c4a2c78>, line 17)

In [16]:
def create_one_parser_model(gold_senses,pred_senses):
    sense_model = dict()
    
    sense_set = set(gold_senses)
    sense_compare_zip = zip(*[gold_senses,pred_senses])
    sense_counter = Counter(sense_compare_zip)
    pred_counter = Counter(pred_senses)
    gold_counter = Counter(gold_senses)
    
    for sense in sense_set:
        gold_total = gold_counter[sense]
        pred_total = pred_counter[sense]
        
        prec,rec,f1 = calc_prob_measure(gold_total,pred_total,sense_counter[(sense,sense)])
        sense_model[sense] = {
            "prec":prec,
            "rec":rec,
            "f1":f1
        }
    
    return sense_model
    
    

In [56]:
def weight_parser_by_accuracy(sense_model,gold,parsers):
    
    parser_f1_scores = []
    
    for name,relations in parsers:
        parser_f1_scores += [(name,scorer.evaluate_sense(gold,relations).compute_micro_average_f1()[0])]
    parser_f1_sum = sum(zip(*parser_f1_scores)[1])
    parser_weights = {name:f1_score/parser_f1_sum for name,f1_score in parser_f1_scores}
    
    for tmp_model in sense_model:
        tmp_model["weight"] = parser_weights[tmp_model["parser"]]
        
    return sense_model

In [32]:
gold_senses,parser_senses,parser_names = get_sense_lists(total_alignments)

In [59]:
model = []
model_path = "data/project_files/test/sense_model_V2.json"

for parser_name,parser_pred in zip(*[parser_names,parser_senses]):
    tmp_model = create_one_parser_model(gold_senses,parser_pred)
    model += [{
        "parser":parser_name,
        "sense_pred": tmp_model
    }]
    


In [60]:
gold_path = "data/gold_standard/test/relations.json"
parsers_path = "data/submissions/sense_only/test/"

In [61]:
model = weight_parser_by_accuracy(
    model,
    read_json(gold_path),
    [(name,read_json(path)) for name,path in get_parser_paths(parsers_path)])

#model
    
save_json(model,model_path)

## Sense Model Statistics

In [358]:
rows = []

for sense in set(gold_senses):
    row = [sense]
    for parser in model:
        parser_senses = parser["sense_pred"]
        if sense in parser_senses:
            row += [parser_senses[sense]["f1"]]
        else:
            row += [0]
    rows += [tuple(row)]


In [359]:
pd.DataFrame(data=rows, columns=["Sense"]+parser_names)

,Sense,steven,oslopots,ecnucs,tao0920,goethe,nguyenlab,clac,PurdueNLP,gw0,ykido,gtnlp
0,Expansion.Conjunction,0.217604,0.337165,0.327660,0.324775,0.323904,0.320285,0.291399,0.305147,0.116766,0.325914,0.324074
1,Temporal.Asynchronous.Precedence,0.252427,0.425000,0.364706,0.382716,0.341772,0.419753,0.381579,0.368421,0.000000,0.378378,0.406977
2,Expansion.Alternative,0.250000,0.400000,0.333333,0.384615,0.444444,0.454545,0.454545,0.400000,0.000000,0.333333,0.454545
3,Temporal.Synchrony,0.170068,0.306250,0.310976,0.300000,0.308176,0.316384,0.298780,0.319527,0.072125,0.319277,0.300000
4,Temporal.Asynchronous.Succession,0.225225,0.369369,0.320388,0.369369,0.342593,0.353448,0.288660,0.311321,0.000000,0.303030,0.303030
5,Contingency.Cause.Result,0.155660,0.222222,0.180488,0.194570,0.225806,0.206422,0.180328,0.171717,0.000000,0.224138,0.213592
6,EntRel,0.194286,0.235686,0.259740,0.269978,0.234568,0.245763,0.200765,0.261053,0.000000,0.173267,0.254576
7,Comparison.Concession,0.074627,0.200000,0.189655,0.225806,0.153846,0.157143,0.140000,0.213115,0.000000,0.200000,0.102564
8,Contingency.Condition,0.293651,0.437500,0.401869,0.411215,0.380952,0.412844,0.370000,0.383178,0.000000,0.427273,0.436975
9,Comparison.Contrast,0.256011,0.366469,0.336620,0.346705,0.324963,0.357955,0.329231,0.337520,0.007264,0.346812,0.352638
